# Overhead images Car counting with deep learning

In [1]:
from keras import layers
import numpy as np
from keras.utils import plot_model
import os
import shutil
import datetime

Using TensorFlow backend.


In [2]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Reshape
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Add
from keras.layers import BatchNormalization
from keras.layers import MaxPool2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dropout
from keras.layers import ReLU
from keras.layers import Softmax
from keras.layers import Dense
from keras.utils import plot_model
from keras.layers.merge import concatenate
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from keras.optimizers import RMSprop, Adam


### Organize data in proper folders

In [3]:
data_folder = 'data'
training_dir = os.path.join(data_folder, 'training')
validation_dir = os.path.join(data_folder, 'validation')

#create training and validation folders
if not os.path.exists(training_dir):
    os.makedirs(training_dir)
    
if not os.path.exists(validation_dir):
    os.makedirs(validation_dir)
    
#inside the training and validation folders we need to create a folder for each class

for i in range(64):
    #path to the ith class training data
    training_class_dir = os.path.join(training_dir, str(i))
    #path to the ith class validation data
    validation_class_dir = os.path.join(validation_dir, str(i))
    
    if not os.path.exists(training_class_dir):
        os.makedirs(training_class_dir)
        
    if not os.path.exists(validation_class_dir):
        os.makedirs(validation_class_dir)


### read images paths (training and testing) and their labels into annotation array 

In [4]:
training_annotations_file = 'COWC_train_list_64_class.txt'
validation_annotations_file = 'COWC_test_list_64_class.txt'

base_dir = 'Columbus_CSUAV_AFRL'
base_training = os.path.join(base_dir, 'train')
base_validataion = os.path.join(base_dir, 'test')

with open(training_annotations_file, 'r') as f:
    training_annotations = [line.strip().split(' ') for line in f.readlines()] 
    training_annotations = [[os.path.join(base_training, pth.split('/')[-1]), cnt] for pth, cnt in training_annotations]

with open(validation_annotations_file, 'r') as f:
    validation_annotations = [line.strip().split(' ') for line in f.readlines()]
    validation_annotations = [[os.path.join(base_validataion, pth.split('/')[-1]), cnt] for pth, cnt in validation_annotations]

### put images in their corresponding folders

In [5]:
for img_path, cnt in training_annotations:
    if os.path.exists(img_path):
        shutil.copy2(img_path, os.path.join(training_dir, cnt))
        
for img_path, cnt in validation_annotations:
    if os.path.exists(img_path):
        shutil.copy2(img_path, os.path.join(validation_dir, cnt))

In [6]:
# data augmentation:
# * small rotations
# * horizontal flip
train_data_gen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=10,
                                   horizontal_flip=True)

validation_data_gen = ImageDataGenerator(rescale=1./255)

#
train_generator = train_data_gen.flow_from_directory(training_dir,
                                                    target_size=(256, 256),
                                                    batch_size=32,
                                                    class_mode='categorical')

validation_generator = validation_data_gen.flow_from_directory(validation_dir,
                                                    target_size=(256, 256),
                                                    batch_size=32,
                                                    class_mode='categorical')

Found 7595 images belonging to 64 classes.
Found 2110 images belonging to 64 classes.


In [7]:
def resception_module(x,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):
    
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu')(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same')(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu')(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same')(conv_5x5)

    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same')(pool_proj)

    conv_1x1 = Conv2D(filters_3x3 + filters_5x5 + filters_pool_proj, (1, 1), padding='same')(x)
    
    conc = concatenate([conv_3x3, conv_5x5, pool_proj], axis=-1, name=name)
    
    output = Add()([conc, conv_1x1])
    
    output = BatchNormalization()(output)
    output = ReLU()(output)
    
    return output

In [8]:
input_layer = Input(shape=(256, 256, 3))

x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu')(input_layer)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)
x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu')(x)
x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu')(x)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)

x = resception_module(x,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32,
                     name='resception_3a')

x = resception_module(x,
                     filters_3x3_reduce=128,
                     filters_3x3=192,
                     filters_5x5_reduce=32,
                     filters_5x5=96,
                     filters_pool_proj=64,
                     name='resception_3b')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)

x = resception_module(x,
                     filters_3x3_reduce=96,
                     filters_3x3=208,
                     filters_5x5_reduce=16,
                     filters_5x5=48,
                     filters_pool_proj=64,
                     name='resception_4a')

x = resception_module(x,
                     filters_3x3_reduce=112,
                     filters_3x3=224,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='resception_4b')

x = resception_module(x,
                     filters_3x3_reduce=128,
                     filters_3x3=256,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='resception_4c')

x = resception_module(x,
                     filters_3x3_reduce=144,
                     filters_3x3=288,
                     filters_5x5_reduce=32,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='resception_4d')

x = resception_module(x,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='resception_4e')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)

x = resception_module(x,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='resception_5a')

x = resception_module(x,
                     filters_3x3_reduce=192,
                     filters_3x3=384,
                     filters_5x5_reduce=48,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='resception_5b')

x = GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)

x = Dropout(0.4)(x)

x = Dense(64, activation='softmax', name='output')(x)
model = Model(input_layer, x, name='resception')

In [9]:
# summarize model
model.summary()

Model: "resception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 64, 64, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 64, 64, 64)   4160        max_pooling2d_1[0][0]            
_________________________________________________________________________________________

In [10]:
def lr_scheduler(epoch, lr):
    if epoch<= 30: return 1e-3
    elif epoch <=60: return 1e-4
    return 1e-5

now = datetime.datetime.now()
time = now.strftime("%d/%m/%Y_%H_%M_%S")

scheduler = LearningRateScheduler(lr_scheduler, verbose=1)
checkpointer = ModelCheckpoint('resception_model_{}.h5'.format(time), monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopper = EarlyStopping(monitor='val_accuracy', patience=5)
###################################################################################################################


model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-3), metrics=['accuracy'])
model.fit_generator(train_generator, epochs=100, validation_data=validation_generator,
                    callbacks=[checkpointer, earlystopper, scheduler])


Epoch 1/100

Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
 15/238 [>.............................] - ETA: 17:24 - loss: 2.8633 - accuracy: 0.2937

KeyboardInterrupt: 

# Fine tuning a pretrained model

In [12]:
from keras.applications.inception_v3 import InceptionV3 
from keras.models import Sequential


In [13]:
base = InceptionV3(input_shape=(256, 256, 3), include_top=False, weights='imagenet')

In [14]:
base.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_58 (Conv2D)              (None, 127, 127, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_10 (BatchNo (None, 127, 127, 32) 96          conv2d_58[0][0]                  
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 127, 127, 32) 0           batch_normalization_10[0][0]     
_______________________________________________________________________________________

In [15]:
classes = os.listdir(training_dir)
n_classes = len(classes)

model = Sequential([
    base,
    GlobalAveragePooling2D(),
    Reshape((1, 1, 2048)),
    Dropout(0.3),
    Conv2D(n_classes, (1, 1)),
    Reshape((n_classes,)),
    Softmax()
])

In [16]:
now = datetime.datetime.now()
time = now.strftime("%d/%m/%Y_%H_%M_%S")

scheduler = LearningRateScheduler(lr_scheduler, verbose=1)
checkpointer = ModelCheckpoint('inception_v3_model_{}.h5'.format(time), monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopper = EarlyStopping(monitor='val_accuracy', patience=5)


### First we will fine tune the new added layers (their parameters are completely random when they were added so we need to polish them a bit)

In [17]:
# Now we will freeze everything in the base model and keep the classification layer
for layer in base.layers:
    layer.trainable = False

In [18]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-3), metrics=['accuracy'])
model.fit_generator(train_generator, epochs=100, validation_data=validation_generator,
                    callbacks=[checkpointer, earlystopper, scheduler])

Epoch 1/100

Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
  3/238 [..............................] - ETA: 21:39 - loss: 3.7402 - accuracy: 0.0938   

KeyboardInterrupt: 

### Fine tune more layers

In [19]:
# Now we will freeze everything in the base model but the last 31 layers
for layer in base.layers[-31:]:
    layer.trainable = True


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-3), metrics=['accuracy'])
model.fit_generator(train_generator, epochs=10, validation_data=validation_generator,
                    callbacks=[checkpointer, earlystopper, scheduler])